## Cluster the Neighbourhoods

In [1]:
import pandas as pd
import numpy as np
import folium

In [6]:
df_toronto = pd.read_csv('df_tornto_coords.csv').drop(df_toronto.columns[0],axis=1)

In [7]:
df_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
1,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
2,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor",43.628841,-79.520999
3,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
4,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849


In [10]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [11]:
# @hiddel_cell
CLIENT_ID = 'CLHA3A43TY2K4K3G1USJTE1TMWUHQ02PZ0TWTJWFGUZF3SXR' # your Foursquare ID
CLIENT_SECRET = 'QP0XLL04DISFKR0WAB5Q5VIY5W5TGLBZPOR3FIIQ4RBNDADM' # your Foursquare Secret
VERSION = '20181212'



In [13]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849
1,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
2,M1G,Scarborough,Woburn,43.770992,-79.216917
3,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
4,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302


### Visualize the Scarborough and its Neighbourhood's

In [14]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [16]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=10)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

###### Top 100 venues in the neighborhood 'Steeles West', from Scarborough

In [17]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Maryvale, Wexford" are 43.7500715, -79.2958491.


In [18]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [20]:
import requests

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c10ca054434b96b6f789fba'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 41,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
        'distance

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
2,DAVIDsTEA,Tea Room,43.776613,-79.258516
3,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
4,SEPHORA,Cosmetics Shop,43.775592,-79.258242
5,Hot Topic,Clothing Store,43.775450,-79.257929
6,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
7,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
8,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
9,Scarborough Town Centre,Shopping Mall,43.775545,-79.257545


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Venues those are Neighborhood to Scaraborough

In [28]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Maryvale, Wexford
Upper Rouge
Woburn
Dorset Park, Scarborough Town Centre, Wexford Heights
Tam O'Shanter
Cedarbrae
Steeles West
Rouge, Malvern
Clairlea, Golden Mile, Oakridge
Highland Creek, Rouge Hill, Port Union
Morningside, West Hill
Ionview, Kennedy Park
Birch Cliff
Cliffcrest, Cliffside
Scarborough Village
Agincourt North, Milliken
Agincourt


In [29]:
scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Maryvale, Wexford",43.750071,-79.295849,Wexford Restaurant,43.746030,-79.293843,Breakfast Spot
1,"Maryvale, Wexford",43.750071,-79.295849,Subway,43.746008,-79.293238,Sandwich Place
2,"Maryvale, Wexford",43.750071,-79.295849,Lebanese bakery,43.746701,-79.292896,Middle Eastern Restaurant
3,"Maryvale, Wexford",43.750071,-79.295849,Wexford Heights Plaza,43.746136,-79.293782,Shopping Mall
4,"Maryvale, Wexford",43.750071,-79.295849,Scarborough Garage Door Repair,43.751288,-79.301508,Auto Garage
5,"Maryvale, Wexford",43.750071,-79.295849,Print Fast,43.746427,-79.292583,Paper / Office Supplies Store
6,"Maryvale, Wexford",43.750071,-79.295849,Sequoia Lounge,43.745645,-79.295737,Middle Eastern Restaurant
7,"Maryvale, Wexford",43.750071,-79.295849,Crown Pastries,43.746098,-79.293142,Bakery
8,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
9,Woburn,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop


In [30]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",4,4,4,4,4,4
Birch Cliff,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Cliffcrest, Cliffside",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"Highland Creek, Rouge Hill, Port Union",2,2,2,2,2,2
"Ionview, Kennedy Park",6,6,6,6,6,6


In [31]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 53 uniques categories.


In [32]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Maryvale, Wexford",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,"Maryvale, Wexford",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,"Maryvale, Wexford",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Maryvale, Wexford",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,"Maryvale, Wexford",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head()

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,...,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.00,0.000,0.0,0.000000,0.000,0.0,0.25,0.000000,...,0.0,0.00,0.0,0.25,0.0,0.25,0.000000,0.000,0.0,0.0
1,"Agincourt North, Milliken",0.0,0.25,0.000,0.0,0.000000,0.000,0.0,0.00,0.000000,...,0.0,0.25,0.0,0.00,0.0,0.00,0.000000,0.000,0.0,0.0
2,Birch Cliff,0.0,0.00,0.000,0.0,0.000000,0.000,0.0,0.00,0.000000,...,0.0,0.00,0.0,0.00,0.0,0.25,0.000000,0.000,0.0,0.0
3,Cedarbrae,0.0,0.00,0.125,0.0,0.125000,0.125,0.0,0.00,0.000000,...,0.0,0.00,0.0,0.00,0.0,0.00,0.000000,0.125,0.0,0.0
4,"Clairlea, Golden Mile, Oakridge",0.0,0.00,0.000,0.0,0.222222,0.000,0.0,0.00,0.222222,...,0.0,0.00,0.0,0.00,0.0,0.00,0.111111,0.000,0.0,0.0


In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Sandwich Place,Breakfast Spot,Lounge,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North, Milliken",Park,Asian Restaurant,Playground,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,Birch Cliff,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Metro Station,Vietnamese Restaurant,College Stadium,General Entertainment
5,"Cliffcrest, Cliffside",American Restaurant,Motel,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
7,"Highland Creek, Rouge Hill, Port Union",Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
8,"Ionview, Kennedy Park",Chinese Restaurant,Hobby Shop,Discount Store,Department Store,Coffee Shop,Train Station,Bank,Convenience Store,Hakka Restaurant,Grocery Store
9,"Maryvale, Wexford",Middle Eastern Restaurant,Auto Garage,Shopping Mall,Sandwich Place,Bakery,Paper / Office Supplies Store,Breakfast Spot,Department Store,College Stadium,Construction & Landscaping


## Now run the K-means clustering with 5 clusters

In [37]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([2, 0, 2, 2, 2, 4, 1, 2, 2, 2])

### Merge the kmeans labels to the original scarborough dataframe

In [38]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1R,Scarborough,"Maryvale, Wexford",43.750071,-79.295849,2,Middle Eastern Restaurant,Auto Garage,Shopping Mall,Sandwich Place,Bakery,Paper / Office Supplies Store,Breakfast Spot,Department Store,College Stadium,Construction & Landscaping
1,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
3,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,2,Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302,2,Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Shopping Mall,Italian Restaurant,Chinese Restaurant,Caribbean Restaurant,Electronics Store
5,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment
6,M1W,Scarborough,Steeles West,43.799525,-79.318389,1,Chinese Restaurant,Fast Food Restaurant,Noodle House,Grocery Store,Breakfast Spot,Pharmacy,Pizza Place,Coffee Shop,Japanese Restaurant,Sandwich Place
7,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store
8,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,2,Bakery,Bus Line,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Metro Station,Vietnamese Restaurant,College Stadium,General Entertainment
9,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


### Visualize the Cluster

In [39]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the each of the 5 clusters

In [40]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Scarborough,0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [41]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,1,Chinese Restaurant,Fast Food Restaurant,Noodle House,Grocery Store,Breakfast Spot,Pharmacy,Pizza Place,Coffee Shop,Japanese Restaurant,Sandwich Place


In [42]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]    

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Middle Eastern Restaurant,Auto Garage,Shopping Mall,Sandwich Place,Bakery,Paper / Office Supplies Store,Breakfast Spot,Department Store,College Stadium,Construction & Landscaping
2,Scarborough,2,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
3,Scarborough,2,Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Shopping Mall,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Scarborough,2,Pizza Place,Noodle House,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Shopping Mall,Italian Restaurant,Chinese Restaurant,Caribbean Restaurant,Electronics Store
7,Scarborough,2,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store
8,Scarborough,2,Bakery,Bus Line,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Metro Station,Vietnamese Restaurant,College Stadium,General Entertainment
9,Scarborough,2,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
10,Scarborough,2,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,2,American Restaurant,Motel,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
14,Scarborough,2,Playground,Convenience Store,Vietnamese Restaurant,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [43]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,3,Chinese Restaurant,Hobby Shop,Discount Store,Department Store,Coffee Shop,Train Station,Bank,Convenience Store,Hakka Restaurant,Grocery Store


In [44]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,4,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bakery,Bank,Lounge,Hakka Restaurant,College Stadium,General Entertainment


## It is obsereved that we have more common neghborhood's in the 3rd cluster with label 2